In [1]:
import pandas as pd
import numpy as np
import datetime as dt

In [2]:
df = pd.read_csv("Downloads/retail sales/customer_shopping_data.csv")

In [3]:
df.head()

,invoice_no,customer_id,gender,age,category,quantity,price,payment_method,invoice_date,shopping_mall
0,I138884,C241288,Female,28,Clothing,5,1500.40,Credit Card,5/8/2022,Kanyon
1,I317333,C111565,Male,21,Shoes,3,1800.51,Debit Card,12/12/2021,Forum Istanbul
2,I127801,C266599,Male,20,Clothing,1,300.08,Cash,9/11/2021,Metrocity
3,I173702,C988172,Female,66,Shoes,5,3000.85,Credit Card,16/05/2021,Metropol AVM
4,I337046,C189076,Female,53,Books,4,60.60,Cash,24/10/2021,Kanyon


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99457 entries, 0 to 99456
Data columns (total 10 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   invoice_no      99457 non-null  object 
 1   customer_id     99457 non-null  object 
 2   gender          99457 non-null  object 
 3   age             99457 non-null  int64  
 4   category        99457 non-null  object 
 5   quantity        99457 non-null  int64  
 6   price           99457 non-null  float64
 7   payment_method  99457 non-null  object 
 8   invoice_date    99457 non-null  object 
 9   shopping_mall   99457 non-null  object 
dtypes: float64(1), int64(2), object(7)
memory usage: 7.6+ MB


In [5]:
df['amount'] = df['quantity'] * df['price']

In [6]:
df['date'] = pd.to_datetime(df['invoice_date']).dt.date.astype('datetime64')

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99457 entries, 0 to 99456
Data columns (total 12 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   invoice_no      99457 non-null  object        
 1   customer_id     99457 non-null  object        
 2   gender          99457 non-null  object        
 3   age             99457 non-null  int64         
 4   category        99457 non-null  object        
 5   quantity        99457 non-null  int64         
 6   price           99457 non-null  float64       
 7   payment_method  99457 non-null  object        
 8   invoice_date    99457 non-null  object        
 9   shopping_mall   99457 non-null  object        
 10  amount          99457 non-null  float64       
 11  date            99457 non-null  datetime64[ns]
dtypes: datetime64[ns](1), float64(2), int64(2), object(7)
memory usage: 9.1+ MB


In [50]:
recency_df = df.groupby(['customer_id'],as_index=False)['date'].max()
recency_df.columns = ['customer_id','LastPurchaseDate']
recency_df.head()

,customer_id,LastPurchaseDate
0,C100004,2021-11-26
1,C100005,2023-03-03
2,C100006,2022-01-12
3,C100012,2021-08-15
4,C100019,2021-07-25


In [51]:
recency_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 99457 entries, 0 to 99456
Data columns (total 2 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   customer_id       99457 non-null  object        
 1   LastPurchaseDate  99457 non-null  datetime64[ns]
dtypes: datetime64[ns](1), object(1)
memory usage: 2.3+ MB


In [52]:
today = recency_df['LastPurchaseDate'].max()
today

Timestamp('2023-12-02 00:00:00')

In [53]:
recency_df['Recency'] = (today - recency_df['LastPurchaseDate']).dt.days
recency_df

,customer_id,LastPurchaseDate,Recency
0,C100004,2021-11-26,736
1,C100005,2023-03-03,274
2,C100006,2022-01-12,689
3,C100012,2021-08-15,839
4,C100019,2021-07-25,860
...,...,...,...
99452,C999886,2023-08-03,121
99453,C999910,2022-05-11,570
99454,C999974,2022-10-05,423
99455,C999976,2022-06-20,530


In [55]:
recency_df.drop(columns=['LastPurchaseDate'],inplace=True)

KeyError: "['LastPurchaseDate'] not found in axis"

In [56]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99457 entries, 0 to 99456
Data columns (total 12 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   invoice_no      99457 non-null  object        
 1   customer_id     99457 non-null  object        
 2   gender          99457 non-null  object        
 3   age             99457 non-null  int64         
 4   category        99457 non-null  object        
 5   quantity        99457 non-null  int64         
 6   price           99457 non-null  float64       
 7   payment_method  99457 non-null  object        
 8   invoice_date    99457 non-null  object        
 9   shopping_mall   99457 non-null  object        
 10  amount          99457 non-null  float64       
 11  date            99457 non-null  datetime64[ns]
dtypes: datetime64[ns](1), float64(2), int64(2), object(7)
memory usage: 9.1+ MB


In [79]:
frequency_df = df.copy()
frequency_df.drop_duplicates(subset=['customer_id','invoice_no'], keep="first", inplace=True) 
frequency_df = frequency_df.groupby('customer_id',as_index=False)['invoice_no'].count()
frequency_df.columns = ['customer_id','Frequency']
frequency_df.head()

,customer_id,Frequency
0,C100004,1
1,C100005,1
2,C100006,1
3,C100012,1
4,C100019,1


In [80]:
monetary_df=df.groupby('customer_id',as_index=False)['amount'].sum()
monetary_df.columns = ['customer_id','Monetary']
monetary_df.head()

,customer_id,Monetary
0,C100004,7502.00
1,C100005,2400.68
2,C100006,322.56
3,C100012,130.75
4,C100019,35.84


In [81]:
#combine first recency and frequency..
rf = recency_df.merge(frequency_df,left_on='customer_id',right_on='customer_id')

#combibe rf frame with monetary values..

rfm = rf.merge(monetary_df,left_on='customer_id',right_on='customer_id')

rfm.set_index('customer_id',inplace=True)

rfm.head()

,Recency,Frequency,Monetary
customer_id,,,
C100004,736,1,7502.00
C100005,274,1,2400.68
C100006,689,1,322.56
C100012,839,1,130.75
C100019,860,1,35.84
